# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [6]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [7]:
import getpass
import pyspark
from datetime import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
import math
import helpers

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize the `SparkSession`

In [8]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-rbsteinm>

In [9]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [10]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [42]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [43]:
metadata.show(5)

+----------------+----------+---------+---------+------+
|            name|station_ID|     long|      lat|height|
+----------------+----------+---------+---------+------+
|       Bucuresti|         2|26.074412| 44.44677|     0|
|          Calais|         3| 1.811446|50.901549|     0|
|      Canterbury|         4| 1.075329|51.284212|     0|
|          Exeter|         5|-3.543547|50.729172|     0|
|Fideris, Bahnhof|         7| 9.733756|46.922368|   744|
+----------------+----------+---------+---------+------+
only showing top 5 rows



We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [13]:
metadata.count()

25935

In [44]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [45]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [46]:
pandas_df.distance_to_zh.max()

9.9833358343820819

In [47]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)

### Cleaning main dataset

In [18]:
# load full data
raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
# raw_df = spark.read.load('/datasets/project/istdaten/2017/10', format='csv', header='true', inferSchema='true', sep=';')

In [19]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'driving_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])
df.show(5)

+----------+-------------------+--------------+--------+----------+---------------+-----------+------------+----------------+-------------------+-------------------+----------------+-------------------+-------------------+------------+
|      date|         driving_id|transport_type|train_id|train_type|additional_trip|trip_failed|   stop_name|schedule_arrival|       real_arrival|arr_forecast_status|    schedule_dep|           real_dep|dep_forecast_status|no_stop_here|
+----------+-------------------+--------------+--------+----------+---------------+-----------+------------+----------------+-------------------+-------------------+----------------+-------------------+-------------------+------------+
|13.09.2017|80:06____:17010:000|           Zug|   17010|        RE|          false|      false|Basel Bad Bf|            null|               null|           PROGNOSE|13.09.2017 05:45|13.09.2017 05:45:00|           PROGNOSE|       false|
|13.09.2017|80:06____:17012:000|           Zug|   17012|

In [20]:
#df.select('stop_name').distinct().count()

In [29]:
metadata.select('name').distinct().count()

1040

In [32]:
type(pandas_df.name.unique())

numpy.ndarray

In [49]:
df2 = df.where(df.stop_name.isin(list(pandas_df.name.unique())))

In [50]:
df2.count()

51571541

In [48]:
list(pandas_df.name.unique())

['Zimmerberg-Basistunnel',
 'Urdorf',
 'Birmensdorf ZH',
 'Bonstetten-Wettswil',
 'Urdorf Weihermatt',
 'Waldegg, Birmensdorferstrasse',
 'Zürich, Goldbrunnenplatz',
 'Aesch ZH, Gemeindehaus',
 'Bonstetten, Dorfplatz',
 'Birmensdorf ZH, Zentrum',
 'Zürich HB',
 'Zürich Altstetten',
 'Zürich Stadelhofen',
 'Zürich Tiefenbrunnen',
 'Zürich Oerlikon',
 'Zürich Seebach',
 'Zürich Affoltern',
 'Zürich Wollishofen',
 'Zürich Enge',
 'Zürich Wiedikon',
 'Zürich Wipkingen',
 'Zürich Flughafen',
 'Zürich Hardbrücke',
 'Zürich Binz',
 'Zürich Friesenberg',
 'Zürich Schweighof',
 'Zürich Triemli',
 'Uitikon Waldegg',
 'Ringlikon',
 'Uetliberg',
 'Zürich Stadelhofen FB',
 'Zumikon',
 'Zollikerberg',
 'Maiacher',
 'Zürich Rehalp',
 'Neue Forch',
 'Waltikon',
 'Spital Zollikerberg',
 'Waldburg',
 'Felsenegg',
 'Adliswil (Luftseilbahnstation)',
 'Zürich, Römerhof',
 'Zürich, Dolder',
 'Zürich Brunau',
 'Zürich Saalsporthalle',
 'Zürich HB SZU',
 'Wildpark-Höfli',
 'Zürich Selnau',
 'Zürich Giesshübel

In [54]:
df2.select('stop_name').distinct().count()

972